In [66]:
import pandas as pd
import re

In [67]:
df = pd.read_excel('Official_Data.xlsx')
df = df.drop(['Unnamed: 0','id', 'sample_1_text','sample_2_text','sample_3_text','sample_4_text','sample_5_text',
              'sample_6_text','sample_7_text','sample_8_text', 'id.1', 'sample_options_id', 'start_time', 'end_time'], axis=1)

In [73]:
print(df.columns)
print(df.shape)
print('na value count', df.isna().sum().sum())

Index(['age', 'gender', 'ethnicity', 'race', 'education', 'marital_status',
       'income', 'employment', 'language', 'disability', 'states', 'sample_1',
       'sample_2', 'sample_3', 'sample_4', 'sample_5', 'sample_6', 'sample_7',
       'sample_8'],
      dtype='object')
(15, 19)
na value count 2


In [78]:
# Check for NaN values in the entire DataFrame
print(df.isna().sum())
print(df.dtypes)

age               0
gender            0
ethnicity         0
race              2
education         0
marital_status    0
income            0
employment        0
language          0
disability        0
states            0
sample_1          0
sample_2          0
sample_3          0
sample_4          0
sample_5          0
sample_6          0
sample_7          0
sample_8          0
dtype: int64
age               object
gender            object
ethnicity         object
race              object
education         object
marital_status    object
income            object
employment        object
language          object
disability        object
states            object
sample_1          object
sample_2          object
sample_3          object
sample_4          object
sample_5          object
sample_6          object
sample_7          object
sample_8          object
dtype: object


In [84]:
df.head(3)

,age,gender,ethnicity,race,education,marital_status,income,employment,language,disability,states,sample_1,sample_2,sample_3,sample_4,sample_5,sample_6,sample_7,sample_8,Gender-Marital Status
0,45-54,female,non hispanic,native american,High School,na,"$25,000 - $49,999",Employed Part time,both,i do not have a disability,indiana,Persuasive,Simplier,Empathetic,Persuasive,Original,Original,Persuasive,Original,female-na
1,18-24,male,hispanic,white,High School,single,"Less than $25,000",Employed Part time,english,i do not have a disability,illinois,Original,Simplier,Empathetic,Simplier,Simplier,Original,Original,Persuasive,male-single
2,25-34,female,non hispanic,multiracial,Associate,single,"Less than $25,000",Student,english,i do not have a disability,new York,Original,Original,Simplier,Simplier,Empathetic,Empathetic,Empathetic,Simplier,female-single


In [85]:
melted_data = df.copy()

keep_columns = ['age', 'gender', 'ethnicity', 'race', 'education', 'marital_status', 'income', 'employment', 'language', 'disability']

melted_data = pd.melt(melted_data, id_vars=keep_columns, value_vars=['sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5', 'sample_6', 'sample_7', 'sample_8'],
                      var_name='sample_number', value_name='choice')

melted_data['sample_number'] = melted_data['sample_number'].str.split('_').str[1].astype(int)

sample_numbers = pd.DataFrame({'sample_number': range(1, 9)})

final_data = melted_data.merge(sample_numbers, on='sample_number')

final_data = final_data.drop('sample_number', axis=1)
final_data.head(12)


,age,gender,ethnicity,race,education,marital_status,income,employment,language,disability,choice
0,45-54,female,non hispanic,native american,High School,na,"$25,000 - $49,999",Employed Part time,both,i do not have a disability,Persuasive
1,18-24,male,hispanic,white,High School,single,"Less than $25,000",Employed Part time,english,i do not have a disability,Original
2,25-34,female,non hispanic,multiracial,Associate,single,"Less than $25,000",Student,english,i do not have a disability,Original
3,25-34,female,hispanic,na,Associate,single,"Less than $25,000",Employed Part time,both,i do not have a disability,Empathetic
4,18-24,male,hispanic,native american,Bachelor,single,"$25,000 - $49,999",Student,both,na,Empathetic
5,25-34,female,non hispanic,white,Master,married,"$50,000 - $74,999",Employed Part time,english,i do not have a disability,Original
6,18-24,female,hispanic,white,Associate,single,na,Student,both,i do not have a disability,Empathetic
7,25-34,non binary,hispanic,Puerto Rican,Associate,divorced,"$50,000 - $74,999",Employed Part time,both,na,Original
8,25-34,female,hispanic,na,Bachelor,single,"Less than $25,000",Student,both,i do not have a disability,Original
9,25-34,female,hispanic,NaN,Associate,single,"$100,000 - $149,999",Student,both,i do not have a disability,Empathetic


In [86]:
final_data.shape

(120, 11)

In [87]:
def clean_text(text):
    if pd.isna(text) or str(text).strip().lower() == 'na' or not str(text).strip():
        cleaned_text = 'undisclosed'
    else:
        pattern = r'[^a-zA-Z0-9\s]'  # Keep letters, numbers, and spaces
        cleaned_text = re.sub(pattern, '', str(text))
    return cleaned_text
def clean_income(income):
    if income.strip().lower() == 'na' or not income.strip():
        cleaned_income = 'undisclosed'
    else:
        cleaned_income = re.sub(r'[,$]', '', income)  # Remove commas and dollar signs
    return cleaned_income
def disability_status(disability_status):
    if disability_status.lower() == "i do not have a disability":
        return "no disability"
    elif disability_status.lower() == "i have a disability":
        return "has disability"
    elif disability_status.lower() == "na":
        return "undisclosed"
    else:
        return disability_status
    

In [88]:
final_data.loc[:, 'ethnicity'] = final_data['ethnicity'].apply(clean_text)
final_data.loc[:, 'gender'] = final_data['gender'].apply(clean_text)
final_data.loc[:, 'race'] = final_data['race'].apply(clean_text)
final_data.loc[:, 'education'] = final_data['education'].apply(clean_text)
final_data.loc[:, 'marital_status'] = final_data['marital_status'].apply(clean_text)
final_data.loc[:, 'employment'] = final_data['employment'].apply(clean_text)
final_data.loc[:, 'language'] = final_data['language'].apply(clean_text)
final_data.loc[:, 'disability'] = final_data['disability'].apply(disability_status)
final_data.loc[:, 'income'] = final_data['income'].apply(clean_income)

In [90]:
# Creating interaction variable: 'Gender-Marital Status'
final_data['Gender-Marital Status'] = final_data['gender'] + '-' + final_data['marital_status']

In [92]:
final_data.drop(['gender', 'marital_status'], axis=1, inplace=True)

In [99]:
def encode_income(income_category):
    income_encoding = {
        'Less than 25000': 0,
        '25000 - 49999': 1,
        '50000 - 74999': 2,
        '75000 - 99999': 3,
        '100000 - 149999': 4,
        '150000 or more': 5,
        'prefer not to say': 6
    }
    
    return income_encoding.get(income_category, -1)


def encode_age(age_group):
    age_encoding = {
        '18-24': 0,
        '25-34': 1,
        '35-44': 2,
        '45-54': 3,
        '55-64': 4,
        '65 and above': 5,
        'Prefer not to say': 6
    }
    
    return age_encoding.get(age_group, -1)

In [101]:
final_data['income'] = final_data['income'].apply(encode_income)
final_data['age'] = final_data['age'].apply(encode_age)

In [102]:
final_data.head(3)

,age,ethnicity,race,education,income,employment,language,disability,choice,Gender-Marital Status
0,3,non hispanic,native american,High School,1,Employed Part time,both,no disability,Persuasive,female-undisclosed
1,0,hispanic,white,High School,0,Employed Part time,english,no disability,Original,male-single
2,1,non hispanic,multiracial,Associate,0,Student,english,no disability,Original,female-single


In [103]:
encoded_data = pd.get_dummies(final_data, drop_first=True)

In [104]:
encoded_data.shape

(120, 26)

In [96]:
encoded_data

,age_25-34,age_45-54,ethnicity_non hispanic,race_Puerto Rican,race_multiracial,race_native american,race_undisclosed,race_white,education_Bachelor,education_Doctorate,...,language_spanish,disability_undisclosed,choice_Original,choice_Persuasive,choice_Simplier,Gender-Marital Status_female-single,Gender-Marital Status_female-undisclosed,Gender-Marital Status_male-married,Gender-Marital Status_male-single,Gender-Marital Status_non binary-divorced
0,0,1,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
116,0,0,0,0,0,0,1,0,1,0,...,1,0,0,1,0,1,0,0,0,0
117,0,1,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,1,0,0
118,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [98]:
encoded_data.columns

Index(['age_25-34', 'age_45-54', 'ethnicity_non hispanic',
       'race_Puerto Rican ', 'race_multiracial', 'race_native american',
       'race_undisclosed', 'race_white', 'education_Bachelor',
       'education_Doctorate', 'education_High School', 'education_Master',
       'education_less than high school', 'income_25000 - 49999',
       'income_50000 - 74999', 'income_Less than 25000', 'income_undisclosed',
       'employment_Employed Part time', 'employment_Student',
       'language_english', 'language_spanish', 'disability_undisclosed',
       'choice_Original', 'choice_Persuasive', 'choice_Simplier',
       'Gender-Marital Status_female-single',
       'Gender-Marital Status_female-undisclosed',
       'Gender-Marital Status_male-married',
       'Gender-Marital Status_male-single',
       'Gender-Marital Status_non binary-divorced'],
      dtype='object')